In [2]:
import os
import pandas as pd
import sys
sys.path.append('../cytof')

from hyperion_preprocess import cytof_read_data_roi

In [3]:
dir_data = "/archive/DPDS/Xiao_lab/shared/hudanyun_sheng/github/image_cytof_test_data/CODEX/"
for fname in os.listdir(dir_data):
    df = pd.read_csv(os.path.join(dir_data, fname))

In [4]:
import pathlib
from classes import CytofImage, CytofImageTiff

def cytof_read_data_roi(filename, slide="", roi=None, iltype="hwd"):
    """ Read cytof data (.txt file) as a dataframe

    Inputs:
        filename = full filename of the cytof data (path-name-ext)

    Returns:
        df_cytof = dataframe of the cytof data

    :param filename: str
    :return df_cytof: pandas.core.frame.DataFrame
    """
    ext = pathlib.Path(filename).suffix
    assert len(ext) > 0, "Please provide a full file name with extension!"
    assert ext.upper() in ['.TXT', '.CSV', '.TIFF'], "filetypes other than '.txt' or '.tiff' not supported now!"

    if ext in ['.txt', '.csv']: # tabular
        if ext == '.txt':
            df_cytof = pd.read_table(filename)
#             if roi is None:
#                 roi = os.path.basename(filename).split('.txt')[0]
#             # initialize an instance of CytofImage
#             cytof_img = CytofImage(df_cytof, slide=slide, roi=roi, filename=filename)

        elif ext == '.csv':
            df_cytof = pd.read_csv(filename)
        
        if roi is None:
            roi = os.path.basename(filename).split('.txt')[0]
        # initialize an instance of CytofImage
        cytof_img = CytofImage(df_cytof, slide=slide, roi=roi, filename=filename)

    elif ext == '.tiff': # image
        image = skio.imread(filename, plugin="tifffile")
        if iltype != "hwd":
            image = image.transpose(np.argsort(['hwd'.index(x) for x in iltype]))
        cytof_img = CytofImageTiff(image, slide="", roi="", filename="")
    return cytof_img

In [5]:
# read data: file name (.csv file) -> dataframe
cytof_img = cytof_read_data_roi(os.path.join(dir_data, fname))

In [43]:
dict_xy = {"X (X):X (X)": "X", "Y (Y):Y (Y)": "Y"}
list(dict_xy.values())

['X', 'Y']

In [6]:
def set_XY(self, dict_xy):
    """
    Args:
        dict_xy: old -> new
    """
    self.df.rename(columns = dict_xy, inplace = True)

dict_xy = {"X (X):X (X)": "X", "Y (Y):Y (Y)": "Y"}
set_XY(cytof_img, dict_xy)

In [ ]:
# marker-label
pattern  = '(.+) \((.+)\):(.+) \((.+)\)'
metadata = ["protein", "marker","protein", "marker"]
colname  = 'Cyc_2_ch_4 (CD56):Cyc_2_ch_4 (CD56)'


In [39]:
# marker-label
pattern  = '(.+) \((.+)\):(.+) \((.+)\)'
metadata = ["protein", "marker","protein", "marker"]
colname  = 'Cyc_2_ch_4 (CD56):Cyc_2_ch_4 (CD56)'

# pattern  = "(\w+.*)\((\w+\))"# '(w.+*)\((w+)\)'
# # metadata = 
# colname  = 'CD38(Pr141Di)'

re.findall(pattern, colname)

[('Cyc_2_ch_4', 'CD56', 'Cyc_2_ch_4', 'CD56')]

In [7]:
import re
def get_markers(self, imarker0=None, markers=None, pattern="\w+.*\(\w+\)"):
    """ Get (1) channels: a list of channel names correspond to each image channel
            (2) markers: a list of protein markers used to obtain the image
            (3) a list of labels tagged to each of the protein markers
    Args:
        pattern = [None, 'cytof', string]
            None: No filtering
            'cytof': "\w+.*\(\w+\)" filtering
            string: customized regular expression
            TODO: contact hudanyun.sheng@utsouthwestern.edu
        markers: If not None, a list of column names (?)
    """
    if markers is None:
        self.raw_channels = list(self.df.columns).copy()
        self.raw_channels.pop('X')
        self.raw_channels.pop('Y')
    
    if pattern is None:
        ## Channels = markers
        
    elif pattern == 'cytof':
        pattern = "\w+.*\(\w+\)"
        
    self.columns = list(self.df.columns)
    if imarker0 is not None:  # if the index of the 1st marker provided
        self.raw_channels = self.columns[imarker0:]
    else:  # assumption: channel names have the common expression: marker(label*)

        self.raw_channels = [re.findall(pattern, t)[0] for t in self.columns if len(re.findall(pattern, t)) > 0]

    self.raw_markers = [x.split('(')[0] for x in self.raw_channels]
    self.raw_labels = [x.split('(')[-1].split(')')[0] for x in self.raw_channels]

    self.channels = self.raw_channels.copy()
    self.markers  = self.raw_markers.copy()
    self.labels   = self.raw_labels.copy()

In [14]:
re.findall('(.+) \((.+)\):(.+) \((.+)\)', 'Cyc_2_ch_4 (CD56):Cyc_2_ch_4 (CD56)')

[('Cyc_2_ch_4', 'CD56', 'Cyc_2_ch_4', 'CD56')]

In [34]:
import numpy as np
cytof_img.df['File Name'].dtype == 'O'

True

In [22]:
cytof_img.df.head()

,ClusterID,EventID,File Name,Index in File,Event Name,Comment,Cyc_2_ch_4 (CD56):Cyc_2_ch_4 (CD56),Cyc_3_ch_3 (FoxP3):Cyc_3_ch_3 (FoxP3),Cyc_3_ch_4 (CD21):Cyc_3_ch_4 (CD21),Cyc_4_ch_2 (MUC1):Cyc_4_ch_2 (MUC1),...,Cyc_22_ch_3 (CD138):Cyc_22_ch_3 (CD138),Cyc_22_ch_4 (Arginase1):Cyc_22_ch_4 (Arginase1),Cyc_23_ch_3 (CD73):Cyc_23_ch_3 (CD73),Cyc_23_ch_4 (CD206):Cyc_23_ch_4 (CD206),Cyc_24_ch_4 (COX2):Cyc_24_ch_4 (COX2),Cyc_25_ch_3 (MMP9):Cyc_25_ch_3 (MMP9),X,Y,Cyc_2_ch_3 (PDL1):Cyc_2_ch_3 (PDL1),Cyc_10_ch_3 (p53):Cyc_10_ch_3 (p53)
0,2412,7,reg001_Compensated_Z planes,7,7,NaN,208.876328,0.002347,0.000348,-0.000597,...,0.000497,271.666809,261.453430,430.244415,56.049427,-0.000393,3.0,288.0,150.706512,235.618027
1,2412,9,reg001_Compensated_Z planes,9,9,NaN,524.119873,-0.000855,41.845718,-0.001695,...,-0.000725,0.000634,1684.988281,-0.000210,-0.000051,0.001196,0.0,357.0,0.000000,0.000000
2,2412,12,reg001_Compensated_Z planes,12,12,NaN,26.963015,0.000171,284.727112,0.001427,...,0.000697,310.491180,0.000323,343.787781,36.308777,0.000140,0.0,432.0,0.000000,0.000000
3,2412,15,reg001_Compensated_Z planes,15,15,NaN,36.401138,0.000741,98.835175,-0.000244,...,-0.001205,187.544769,-0.000955,0.000148,11.342896,0.000986,0.0,478.0,0.000000,0.000000
4,2412,17,reg001_Compensated_Z planes,17,17,NaN,86.979965,-0.000445,45.013542,-0.000433,...,-0.000173,1435.536377,100.552376,0.000222,57.033249,-0.003303,8.0,205.0,25.728107,1414.393311


In [8]:
## TODO: come up with better regular expression

channel_pattern = "\w+.*\(\w+\):\w+.*\(\w+\)"
get_markers(cytof_img, pattern=channel_pattern)

In [ ]:
def preprocess(self):
    nrow = max(self.df['Y'].values) + 1
    ncol = max(self.df['X'].values) + 1
    n = len(self.df)
    if nrow * ncol > n:
        df2 = pd.DataFrame(np.zeros((nrow * ncol - n, len(self.df.columns)), dtype=int),
                           columns=self.df.columns)
        self.df = pd.concat([self.df, df2])

In [23]:
cytof_img.preprocess()

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [25]:
cytof_img.df['X']

0            3.0
1            0.0
2            0.0
3            0.0
4            8.0
           ...  
192136    8241.0
192137    8346.0
192138    8366.0
192139    9173.0
192140    9312.0
Name: X, Length: 192141, dtype: float64